In [2]:
import xml.etree.ElementTree as ET
import random

In [3]:
def load_network_xml(network_file):
    # Parse the network XML file
    tree = ET.parse(network_file)
    root = tree.getroot()

    # Create a dictionary to store node coordinates by ID
    node_coords = {}

    # Find all node elements
    for node in root.findall(".//node"):
        node_id = node.get("id")
        x = float(node.get("x"))
        y = float(node.get("y"))

        # Store node coordinates in the dictionary
        node_coords[node_id] = (x, y)

    return node_coords

In [17]:
def create_population_and_plans_xml(num_agents, node_coords, output_file_path):
    # Create the root element for the plans
    plans = ET.Element("plans", attrib={'xml:lang': 'de-CH'})  # Root element with lang attribute

    # List of node IDs to randomly select for activities
    node_ids = list(node_coords.keys())

    # Loop over the number of agents
    for i in range(1, num_agents + 1):
        # Create a person element
        person = ET.SubElement(plans, "person", id=str(i))

        # Create a plan for the person
        plan = ET.SubElement(person, "plan", selected="yes")

        # Randomly choose home and work nodes
        home_node_id = random.choice(node_ids)
        work_node_id = random.choice(node_ids)

        home_node = node_coords[home_node_id]
        work_node = node_coords[work_node_id]

        # Define the agent's activities and legs
        home_activity = ET.SubElement(plan, "act", type="h", 
                                      x=str(home_node[0]), y=str(home_node[1]), end_time="08:00:00")
        leg_to_work = ET.SubElement(plan, "leg", mode="car")
        work_activity = ET.SubElement(plan, "act", type="w", 
                                      x=str(work_node[0]), y=str(work_node[1]), start_time="08:30:00", end_time="17:00:00")
        leg_to_home = ET.SubElement(plan, "leg", mode="car")
        home_activity_2 = ET.SubElement(plan, "act", type="h", 
                                        x=str(home_node[0]), y=str(home_node[1]))

    # Convert the ElementTree to a string
    tree = ET.ElementTree(plans)

    # Manually write the header to the output file
    with open(output_file_path, "wb") as f:
        # Write the XML declaration and DOCTYPE
        f.write(b'<?xml version="1.0" ?>\n')
        f.write(b'<!DOCTYPE plans SYSTEM "http://www.matsim.org/files/dtd/plans_v4.dtd">\n')

        # Write the tree structure
        tree.write(f)

In [19]:
node_coords = load_network_xml("../../scenarios/utah/utah_network_cleanv2.xml")

In [20]:
create_population_and_plans_xml(100, node_coords, "../../scenarios/utah/utah_network_plan.xml")